<a href="https://colab.research.google.com/github/KevinW824/CSE-144-Applied-Machine-Learning/blob/main/Summer_21_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# Assignment 2

**DUE: Sunday July 4 at 11:59pm**

Turn in the assignment via Canvas.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Runtime→→Restart runtime) and then run all cells (in the menubar, select Runtime→→Run All).

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
NAME = ""
STUDENT_ID = ""

## Problem 1 - Yield Dataset with Polynomial Regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

This data set of size n = 15 contains measurements of yield from an experiment done at five different temperature levels. The variables are y = yield and x = temperature in degrees Fahrenheit. Download the data from PyDrive.

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1snU4nKrW72fgaEYzWvnrK0iRhsVX2W7N"})
downloaded.GetContentFile('yield.csv')  

In [ ]:
# Create pandas dataframe
data = pd.read_csv('yield.csv')

In [ ]:
# Let's look at the data
data

In [ ]:
# Look at data description
data.describe()

In [ ]:
# Split data in X and y using pandas functionality.
X = data.iloc[:,0] # Create vector of explanatory variables
y = data.iloc[:,1] # Create vector of target variables

#cast dataframes into numpy arrays
X = X.values
y = y.values


# Split data into X_train, Y_train, X_test y_test using sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

# Reshape X_train, X_test for sklearn LinearRegression
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

### a) Linear Regression (10 points)

We will run a simple linear regression on this well curated dataset using sklearn's **LinearRegression**.

Hint: Review the sklearn docs https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html and class exercise.

In [ ]:
from sklearn.linear_model import LinearRegression
# instantiate LinearRegression
regressor = ### YOUR CODE HERE ### 

# Fit the regressor using X_train and y_train
### YOUR CODE HERE ###

Let's now visually inspect the model and see how well this model predicts the yield. First, we will plot the yield vs temp. points from training set in black. Then, we will plot the regression line in red. Predictions from the regression model lie along the red line.

In [ ]:
#Plot the actual yield (y_train) vs temp. values from training set
plt.scatter(X_train, y_train,  color='black', label = "yield")

# Plot the regression line
x = np.linspace(50, 100,15) # Generate 15 points to plot our regression line
x = x.reshape(-1,1) # reshape data for our regression
y = regressor.predict(x) # feed x's into regressor
plt.plot(x, y, color='red', linewidth=3, label = "regression line") # plot the regression line

plt.legend(loc='lower right')
plt.title('Training set yield vs. temp With Regression Line')
plt.show()

Let's also visually inspect the prediction results from the test set.

In [ ]:
#Plot the actual yield (y_train) vs temp. values  for the test set
plt.scatter(X_test, y_test,  color='black', label = "yield")

# Plot the regression line
x = np.linspace(50, 100,15) # Generate 15 points to plot our regression line
x = x.reshape(-1,1) # reshape data for our regression
y = regressor.predict(x) # feed x's into regressor
plt.plot(x, y, color='red', linewidth=3, label = "regression line") # plot the regression line

plt.legend(loc='lower right')
plt.title('Test set yield vs. temp with regression line')
plt.show()

### b) Inspecting linear regression model (5 points)

After inspecting the results visually, does this model appear to be a good or bad fit on the test set and training set? 

[YOUR ANSWER HERE]

Does the model display signs of underfitting? If so why?

[YOUR ANSWER HERE]

### c) Polynomial Regression (10 points)

Data may not follow a linear relationship from the independent variable $X$ to the dependent variable $y$. Fitting a linear model to this would be inaccurate and yield a high loss. 

If we want to model an order $d$ polynomial relationship between $X$ and $y$ we can augment our initial linear model where instead of having:
$$
y_i = \theta_0 + \theta_1 x_i
$$

We have:

$$
y_i = \theta_0 + \theta_1 x_i + \theta_2 x_i^2 + \cdots + \theta_d x_i^d
$$

We can use the same linear regression algorithm we if we first augment $X$ and add extra columns (or dimensions). 

$$ \textbf X =
\begin{bmatrix}
    x_{1}       & x_{1}^2 & \cdots & x_{1}^d \\
    x_{2}       & x_{2}^2 & \cdots & x_{2}^d \\
    \vdots       & \vdots & \ddots & \vdots \\
    x_{n}       & x_{n}^2 & \cdots & x_{n}^d
\end{bmatrix}$$

Then our new higher order $\hat y$ is computed same as before.

$$ \hat y =  X \theta =
\begin{bmatrix}
    1 & x_{1}       & x_{1}^2 & \cdots & x_{1}^d \\
    1 & x_{2}       & x_{2}^2 & \cdots & x_{2}^d \\
    \vdots & \vdots       & \vdots & \ddots & \vdots \\
    1 & x_{n}       & x_{n}^2 & \cdots & x_{n}^d
\end{bmatrix}
\begin{bmatrix}\theta_0 \\ \theta_1 \\ \vdots \\ \theta_{d} 
\end{bmatrix}=
\begin{bmatrix}
    \theta_0 + \theta_1 x_{1} + \theta_2 x_{1}^2 + \cdots + \theta_{d}  x_{1}^d \\
    \theta_0 + \theta_1 x_{2} + \theta_2 x_{2}^2 + \cdots + \theta_{d}  x_{2}^d  \\
    \vdots   \\
    \theta_0 + \theta_1 x_{n} + \theta_2 x_{n}^2 + \cdots + \theta_{d}  x_{n}^d
\end{bmatrix} 
= \begin{bmatrix}\hat y_1 \\ \hat y_2 \\ \vdots \\ \hat y_{n} 
\end{bmatrix}$$

Using sklearn's **PolynomialFeatures** functionality, we will now transform X_train and X_test into second order polynomial space.

For more info: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [ ]:
# First, lets look at the values of X_train
print(X_train)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

#Create polynomial transform of degree 2
poly = PolynomialFeatures(2)

# Transform X_train X_test
X_train_pol = poly.fit_transform(X_train)
X_test_pol = poly.fit_transform(X_test)

# Print X_train_pol
print(X_train_pol)

We can now fit a second degree polynomial regression model to the transformed data.

In [ ]:
from sklearn.linear_model import LinearRegression
# instantiate LinearRegression
regressor = ### YOUR CODE HERE ### 

# Fit the regressor using X_train_pol and y_train
### YOUR CODE HERE ###

Now, visually inspect the polynomial regression model and see how well this model predicts the yield on the training set. yield vs. temp. points are plotted in black along with the polynomial regression line in red.

In [ ]:
#Plot the actual yield (y_train) vs temp. values from training set
plt.scatter(X_train, y_train,  color='black', label = "yield")

# Plot the regression line
x = np.linspace(50, 100,15) # Generate 15 points to plot our regression line
x = x.reshape(-1,1) # reshape for our regression
y = regressor.predict(poly.fit_transform(x)) # feed transformed x's into regressor
plt.plot(x, y, color='red', label = "regression line") # plot regression line

plt.legend(loc='lower right')
plt.title('Test set yield vs. temp with polynomial regression line')
plt.show()

Lastly, lets plot the test set yield vs. temp. points along with the polynomial regression line.

In [ ]:
#Plot the actual yield (y_train) vs temp. values from training set
plt.scatter(X_test, y_test,  color='black', label = "yield")

# Plot the regression line
x = np.linspace(50, 100,15) # Generate 15 points to plot our regression line
x = x.reshape(-1,1) # reshape for our regression
y = regressor.predict(poly.fit_transform(x)) # feed transformed x's into regressor
plt.plot(x, y, color='red', label = "regression line") # plot regression line

plt.legend(loc='lower right')
plt.title('Test set yield vs. temp with polynomial regression line')
plt.show()

### d) Inpsecting polynomial regression model (5 points)

After inspecting the polynomial regression model, does the model appear to be a better fit than simple linear regression? 

[YOUR ANSWER HERE]

Does the model address display more or less underfitting than the simple linear regression model and why?

[YOUR ANSWER HERE]

## Problem 2 - Fish Dataset

## Importing the dataset

We will now perform different variations of linear regression to predict fish weight given Species, weight, and physical measurements. The different attributes of the data are:

- Species: species name of fish

- Weight: weight of fish in Gram g

- Length1: vertical length in cm

- Length2: diagonal length in cm

- Length3: cross length in cm

- Height: height in cm

- Width: diagonal width in cm

Begin by downloading the Fish.csv file from google drive. Make sure to select your @ucsc.edu email when authorizing access to your account. 

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Download the Fish data
downloaded = drive.CreateFile({'id':"1AtMi-xCejVlhYS5qjgjjW4gH-TLuWJjC"})
downloaded.GetContentFile('Fish.csv')  

In [ ]:
# Create pandas dataframe
fish_data = pd.read_csv('Fish.csv')

Let's look at some of the data and check some of the dataset descriptors.

In [ ]:
# Let's print out the first few rows
fish_data.head()

In [ ]:
# Let's get the data shape
fish_data.shape

In [ ]:
# Let's describe the data
fish_data.describe()

In [ ]:
# Lastly, let's get data info
fish_data.info()

### a) Removing 0's from data (5 points)

We can see that there aren't any null values in this dataset by using the .info() function. However, .describe() shows us there are weights of 0.0g which is ambiguous and is likely a recording error. 

Drop any row that has a weight of 0.0.

In [ ]:
# Delete any rows for which there is a measurement of 0.0.
### YOUR CODE HERE ###

# Let's take another look.
fish_data.describe()

### b) Outlier Detection \& Elimination (5 points)

Using 4 deviations from the mean as our cutoff, and using the data listed using the describe function above, are there any outliers?

[YOUR ANSWER HERE]

## Problem 3 - Fish Linear Regression

You will now run different versions of linear regression in order to predict fish Weight using the 6 explanatory variables.

### a) Multiple Linear Regression (6 points)

First, run a multiple linear regression using only the height, width, and length measurements to predict weight. We will begin by splitting the data into features **X_fish** and target variable **y_fish**.

In [ ]:
y_fish = fish_data.iloc[:, 1] # Get Fish Weights
X_fish = fish_data.drop(columns=['Weight']) # Get Fish measurements plus species
X_fish = X_fish.drop(columns=['Species']) # Drop the Fish Species for now

# print X.head(), you should have 5 features for each sample
print("X_fish.head():")
print(X_fish.head())

# print y.head(), you should have one label for each sample
print("\ny_fish.head()")
print(y_fish.head())

Now, we are almost ready to run regression with scikit-learn. We need to first convert **X_fish** and **y_fish** into numpy arrays and split the data into training and validation splits using sklearn **train_test_split**. 

sklearn's train_test_split offers customizable functionality when creating training and test sets. For more information, checkout https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html .

In [ ]:
# Convert X, and y into np arrays
X_fish = X_fish.values
y_fish = y_fish.values

# Split data into train test split
from sklearn.model_selection import train_test_split
X_fish_train, X_fish_test, y_fish_train, y_fish_test = train_test_split(X_fish, y_fish, test_size = 0.2, random_state = 0)

Now we are ready to run a multiple linear regression. Use sklearn's **LinearRegression** to carry out the regression.

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = ### YOUR CODE HERE ### 

# Fit the regressor using X_fish_train and y_fish_train
### YOUR CODE HERE ###


Now let's see how we did, generate predictions using X_fish_test.

In [ ]:
# generate predictions using X_fish_test
y_fish_pred = ### YOUR CODE HERE ### 

# Print the predictions along with actual weights
np.set_printoptions(precision=2)
print(np.concatenate((y_fish_pred.reshape(len(y_fish_pred),1), y_fish_test.reshape(len(y_fish_test),1)),1))

### b) Computing MSE and R-squared for MLR (5 points)

The model appears to do an okay job predicting some weights while missing the mark on others. Let's quantify the results by computing mean squared error (MSE) and the coefficient of determination (R-squared). sklearn's metrics package provides functions to compute the values for you. 

More info can be found here: 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Compute the mean squared error using y_fish_test and y_fish_pred
### YOUR CODE HERE ###

# Compute the coefficient of determination using y_fish_test and y_fish_pred
### YOUR CODE HERE ###


Using the y_fish_test and y_fish_pred, what was the mean squared error for this model?

[YOUR ANSWER HERE]

Using the y_fish_test and y_fish_pred, what was the coefficient of detemination for this model?

[YOUR ANSWER HERE]

### c) Polynomial Multiple Linear Regression (12 points)

We have seen how polynomial regression can increase the predictive power of linear regression models. We will now run a polynomial multiple linear regression model in order to gain a more accurate model. Begin by transforming the features X_fish_test and X_fish_train into second order polynomial space. There will be interaction terms in the transformed dataset as well. For more on interaction terms visit the sklearn docs.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
#Create polynomial transform of degree 2. hint: problem 1
poly = ### YOUR CODE HERE ###

# Transform X_fish_train X_fish_test to second order polynomial space
X_fish_train_pol = ### YOUR CODE HERE ###
X_fish_test_pol = ### YOUR CODE HERE ###

# Print 5 rows of X_fish_train_poly, shape should be (5,21)
print(X_fish_train_pol[0:5,:])

Next run the second order polynomial regression using the transformed data. Instantiate a new regression model and use X_fish_train_pol and y_fish_train to fit the model.

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = ### YOUR CODE HERE ### 


regressor = LinearRegression()

# Fit the regressor using X_fish_train_pol and y_fish_train
### YOUR CODE HERE ###


Now let's see how we did with the second model, generate predictions using X_fish_test_pol.

In [ ]:
# generate predictions using X_test
y_fish_pol_pred = ### YOUR CODE HERE ### 


y_fish_pol_pred = regressor.predict(X_fish_test_pol)

# Print the predictions along with actual weights
np.set_printoptions(precision=2)
print(np.concatenate((y_fish_pred.reshape(len(y_fish_pol_pred),1), y_fish_test.reshape(len(y_fish_test),1)),1))

### d) Computing MSE and R-squared for Polynomial Regression (5 points)

Now, compute the mean squared error using y_fish_test and y_fish_pol_pred

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Compute the mean squared error using y_fish_test and y_fish_pol_pred
### YOUR CODE HERE ###

# Compute the coefficient of determination using y_fish_test and y_fish_pol_pred
### YOUR CODE HERE ###


Using the y_fish_test and y_fish_pol_pred, what was the mean squared error for this model?

[YOUR ANSWER HERE]

Using the y_fish_test and y_fish_pol_pred, what was the coefficient of detemination for this model?

[YOUR ANSWER HERE]

### e) Multiple Linear Regression With Categorical Variable (12 points)

Now that we have done multiple linear regression, and polynomial multiple linear regression, we will now do a multiple linear regression that takes into account the species of fish by creating numeric levels.

In [ ]:
# Create a copy of fish_data
fish_data_species = fish_data

# Begin by generating a dictionary that maps all unique species in fish_data to a unique id
level_dict = ### YOUR CODE HERE ###

# Uses the level dictionary to retrieve the id
def integer_encode_level(level):
    ### YOUR CODE HERE ###

# Apply the function to the Species column and store in Species column (you should overwrite the species current data with the numeric representations)
fish_data_species['Species'] = ### YOUR CODE HERE ###

# Check data
fish_data_species

We will now create a dataset with 1 target column and 6 feature columns: Species, Length1, Length2, Length3, Height, Width. 

In [ ]:
# Split fish_data into 
y_fish_categorical = fish_data_species.iloc[:, 1] # Get Fish Weights
X_fish_categorical = fish_data_species.drop(columns=['Weight']) # Get Fish measurements plus species

# print X_fish_categorical.head(), you should have 6 features for each sample
print("X_fish_categorical.head():")
print(X_fish_categorical.head())

# print y_fish_categorical.head(), you should have one label for each sample
print("\ny_fish_categorical.head()")
print(y_fish_categorical.head())

Create training and test sets for the fish data with categorical species variable:

In [ ]:
# Convert X, and y into np arrays
X_fish_categorical = X_fish_categorical.values
y_fish_categorical = y_fish_categorical.values

# Split data into train test split
from sklearn.model_selection import train_test_split
X_fish_train_categorical, X_fish_test_categorical, y_fish_train_categorical, y_fish_test_categorical = train_test_split(X_fish_categorical, y_fish_categorical, test_size = 0.2, random_state = 0)

Now we are ready to run a our final multiple linear regression. As a recap, we added a categorical variable to represent the species of fish. We encoded the species as levels, and overwrote the species column with the numeric representations of the species. Now, use sklearn's **LinearRegression** to carry out regression one last time.

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = ### YOUR CODE HERE ### 

# Fit the regressor using X_fish_train_categorical and y_fish_train_categorical
### YOUR CODE HERE ###


Let's check the predictions for this model. Generate predictions using X_fish_test_categorical and compare to the test set weights.

In [ ]:
# generate predictions using X_fish_test_categorical
y_fish_pred_categorical = ### YOUR CODE HERE ### 


# Print the predictions along with actual weights
np.set_printoptions(precision=2)
print(np.concatenate((y_fish_pred.reshape(len(y_fish_pred_categorical),1), y_fish_test.reshape(len(y_fish_test_categorical),1)),1))

### f) Computing MSE and R-squared for MLR with Categorical Variable (5 points)

In [ ]:
# Compute the mean squared error using y_fish_test_categorical and y_fish_pred_categorical
### YOUR CODE HERE ###

# Compute the coefficient of determination using y_fish_test_categorical and y_fish_pred_categorical
### YOUR CODE HERE ###


Using y_fish_test_categorical and y_fish_pred_categorical, what was the mean squared error for this model?

[YOUR ANSWER HERE]

Using y_fish_test_categorical and y_fish_pred_categorical, what was the coefficient of detemination for this model?

[YOUR ANSWER HERE]

### g) Model selection (15 points)

MSE and the coefficient of determination (R-squared) are used to provide quick statistics on how a model performs. In general a lower MSE means that the labels $y$ are close to the regression line. R-squared is always between 0 and 100%. In general, the higher the R-squared, the better the model fits your data. Based upon the MSE and  R-squared values you computed in parts a,b, and c which model fits the data the best? Also, do the numeric predictions generated by your models reflect your hypothesis?

[YOUR ANSWER HERE]